# Explore BL Sessions Data

Here we introduce some useful DataJoint tools to explore the bl19 pipeline

First thing first, import datajoint

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import utility.blob_transformation as bt
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from matplotlib import cm

import pandas as pd

In [3]:
dj.blob.use_32bit_dims = True

## Get handle to DBs

lab         =  researchers & rig DB <br />
subject     =  rats DB <br />
acquisition =  sessions DB

In [4]:
bdata = dj.create_virtual_module('bdatatest', 'bdatatest')
session_key = {'sessid': 882009}
session_data = (bdata.Sessions & session_key).fetch('protocol_data', as_dict=True)
parsed_events = (bdata.ParsedEvents & session_key).fetch(as_dict=True)

[2022-08-16 14:53:14,509][INFO]: Connecting alvaros@datajoint01.pni.princeton.edu:3306
[2022-08-16 14:53:14,819][INFO]: Connected alvaros@datajoint01.pni.princeton.edu:3306


In [11]:
su = bt.transform_blob(session_data[0]['protocol_data'])
su

[{'hits': array([ 1.,  0.,  1.,  1.,  1., nan,  1., nan,  1.,  1., nan,  0., nan,
         nan, nan, nan,  1.,  1.,  0.,  0.,  0., nan,  0., nan,  1.,  1.,
          1., nan,  0.,  1., nan,  0.,  0.,  1., nan,  1.,  1.,  0., nan,
         nan, nan,  1., nan, nan,  1.,  1.,  1., nan, nan, nan, nan,  1.,
          1.,  1.,  1., nan, nan,  1.,  1.,  0.,  1.,  1.,  1., nan,  0.,
          0.,  1.,  1., nan, nan, nan,  1.,  0.,  1.,  1.,  0.,  0., nan,
          1., nan,  1., nan,  1.,  0.,  0., nan, nan,  1., nan, nan,  1.,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,  0.,
         nan,  1.,  1., nan,  1., nan, nan,  1., nan,  1.,  1.,  1., nan,
          1., nan,  1., nan,  

In [12]:
peh = bt.transform_blob(parsed_events[0]['peh'])
df_peh = pd.DataFrame(peh)
df_peh

,pokes,waves,states
0,"{'C': [[391.283604, 391.955704], [392.01200400...","{'direct_reward': [393.011304, 393.06910400000...","{'state_0': [[nan, 280.845204], [393.569804000..."
1,"{'C': [[397.499403, 397.719003], [397.800103, ...","{'direct_reward': [], 'stimulatorwave': [397.5...","{'state_0': [[nan, 395.201303], [403.298703, n..."
2,"{'C': [[472.895615, 472.92401500000005], [472....","{'direct_reward': [474.732615, 474.791915], 's...","{'state_0': [[nan, 403.298915], [475.292615, n..."
3,"{'C': [[478.64200300000005, 479.060803], [479....","{'direct_reward': [480.42260300000004, 480.480...","{'state_0': [[nan, 476.231303], [480.981203000..."
4,"{'C': [[485.225104, 485.347304], [485.43100400...","{'direct_reward': [487.132604, 487.191904], 's...","{'state_0': [[nan, 481.662504], [487.692604, n..."
...,...,...,...
196,"{'C': [], 'L': [], 'R': [], 'starting_state': ...","{'direct_reward': [], 'stimulatorwave': [], 'c...","{'state_0': [[nan, 9537.876812], [9662.8777120..."
197,"{'C': [], 'L': [], 'R': [], 'starting_state': ...","{'direct_reward': [], 'stimulatorwave': [], 'c...","{'state_0': [[nan, 9662.877912], [9787.8788119..."
198,"{'C': [], 'L': [], 'R': [], 'starting_state': ...","{'direct_reward': [], 'stimulatorwave': [], 'c...","{'state_0': [[nan, 9787.879012], [9912.879912,..."
199,"{'C': [], 'L': [], 'R': [], 'starting_state': ...","{'direct_reward': [], 'stimulatorwave': [], 'c...","{'state_0': [[nan, 9912.880113], [10037.881012..."
